In [1]:
%pip install torch pandas torchvision scikit-learn tqdm kaggle timm scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
# upload kaggle.json first.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!rm -rf data

In [2]:
# !apt update -qq
# !apt install -qq unzip
# !kaggle datasets download nirmalsankalana/sugarcane-leaf-disease-dataset
# !unzip -q sugarcane-leaf-disease-dataset.zip -d data
# !kaggle datasets download  pungliyavithika/sugarcane-leaf-disease-classification
!unzip -q sugarcane-leaf-disease-classification.zip -d data

In [3]:
import os
import shutil
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Define base paths
data_root = "data"
images_dir = os.path.join(data_root, "images")
os.makedirs(images_dir, exist_ok=True)

# Collect metadata for each image
dataset = []

for subfolder in os.listdir(data_root):
    subfolder_path = os.path.join(data_root, subfolder)

    # Skip non-directories and the images directory
    if not os.path.isdir(subfolder_path) or subfolder == "images":
        continue

    for image in os.listdir(subfolder_path):
        old_image_path = os.path.join(subfolder_path, image)

        if os.path.isfile(old_image_path):
            base, ext = os.path.splitext(image)
            new_base = f"{subfolder}_{base}"
            new_filename = f"{new_base}{ext}"
            new_image_path = os.path.join(images_dir, new_filename)

            # Ensure uniqueness
            counter = 1
            while os.path.exists(new_image_path):
                new_filename = f"{new_base}_{counter}{ext}"
                new_image_path = os.path.join(images_dir, new_filename)
                counter += 1

            # Move the file
            shutil.move(old_image_path, new_image_path)

            # Record the file metadata
            dataset.append({
                "image_id": new_filename,  # Just filename
                "label": subfolder,
                "original_filename": image
            })

    # Optionally remove the now-empty subfolder
    os.rmdir(subfolder_path)

# Create a DataFrame
df = pd.DataFrame(dataset)

# Encode the labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# Save the dataset to CSV
df.to_csv(os.path.join(data_root, "dataset.csv"), index=False)

# Save label mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)

Label mapping: {'Healthy': 0, 'RedRot': 1, 'RedRust': 2}


In [1]:
# To load the dataset again:
import pandas as pd
df = pd.read_csv('/workspace/data/dataset.csv')

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [5]:
df["label"].value_counts()

0    75
2    75
1    74
Name: label, dtype: int64

In [6]:
import os

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from dataset import Dataset


In [7]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

# Change the path to the directory where the images are stored
# RTC:data/images/RedRot_24.jpg
path = "data/images"
train_dataset = Dataset(train_df, path)
test_dataset = Dataset(test_df, path)
val_dataset = Dataset(val_df, path)

In [ ]:
import itertools

from model import MaiaNet
from train import Trainer

num_epochs = 35
num_classes = 3

def run_experiment(batch_size, lr):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = MaiaNet(num_classes)
    trainer = Trainer(model, train_loader, val_loader, test_loader, lr, num_epochs, batch_size=batch_size)

    trainer.train()
    trainer.test()
    # torch.save(trainer.model.state_dict())
    torch.save(trainer.model.state_dict(), f"maianet_bs{batch_size}_lr{lr}.pth")

batch_size = 8
lr = 2e-4
run_experiment(batch_size, lr)

Epoch 1/35: 100%|██████████| 23/23 [00:12<00:00,  1.90it/s, loss=1.1315]



Train Metrics:
--------------------------------------------------
Epoch: 0
Train Loss: 1.1090
Test Loss: 1.1035
Accuracy: 0.3182
Precision: 0.1012
Recall: 0.3182
F1: 0.1536
--------------------------------------------------


Epoch 2/35: 100%|██████████| 23/23 [00:08<00:00,  2.80it/s, loss=1.1184]



Train Metrics:
--------------------------------------------------
Running experiment with batch_size=8, lr=0.0002
Epoch: 1
Train Loss: 1.1132
Test Loss: 1.0963
Accuracy: 0.3182
Precision: 0.1012
Recall: 0.3182
F1: 0.1536
--------------------------------------------------


Epoch 3/35: 100%|██████████| 23/23 [00:08<00:00,  2.77it/s, loss=1.0887]


...

Epoch 18/35: 100%|██████████| 23/23 [00:08<00:00,  2.75it/s, loss=1.0549]



Train Metrics:
--------------------------------------------------
Epoch: 17
Train Loss: 1.0542
Test Loss: 1.0479
Accuracy: 0.5455
Precision: 0.4000
Recall: 0.5455
F1: 0.4463
--------------------------------------------------


Epoch 19/35: 100%|██████████| 23/23 [00:08<00:00,  2.74it/s, loss=1.0299]



Train Metrics:
--------------------------------------------------
Epoch: 18
Train Loss: 1.0601
Test Loss: 1.0540
Accuracy: 0.4545
Precision: 0.3333
Recall: 0.4545
F1: 0.3719
--------------------------------------------------


Epoch 20/35:  13%|█▎        | 3/23 [00:01<00:07,  2.67it/s, loss=1.0420]

In [13]:
!pip install torchcam -q

In [15]:
import torch
from model import MaiaNet 
model = MaiaNet(num_classes=3)

state_dict = torch.load("maianet_bs8_lr0.0002.pth", map_location="cpu")
model.load_state_dict(state_dict)

model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MaiaNet(
  (head): HeadBlock(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (anti_aliasing_1): AntiAliasingBlock(
    (block1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (down_conversion): Sequential(
      (0): SiLU()
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): SiLU()
    )
    (block2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [17]:
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d) and module.out_channels == 2048:
        print(name, module)

maia_4.conv1.0 Conv2d(1024, 2048, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
maia_4.conv2.0 Conv2d(2048, 2048, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
maia_4.conv3.0 Conv2d(2048, 2048, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
maia_4.residual_conv Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2))


In [19]:
!pip install --upgrade numpy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 134.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchcam 0.4.0 requires numpy<2.0.0,>=1.17.2, but you have numpy 2.2.4 which is incompatible.


In [20]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

from torchcam.methods import GradCAM, GradCAMpp, ScoreCAM
from torchcam.utils import overlay_mask

from PIL import Image
import numpy as np

# Assuming your model is defined and loaded like this:
from model import MaiaNet
# from data import test_dataset  # Or any dataset with images

# ----------------------
# SETUP
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MaiaNet(num_classes=3).to(device)
model.load_state_dict(torch.load("maianet_bs16_lr0.0001.pth", map_location=device))
model.eval()

# ----------------------
# Choose the target layer
# ----------------------
# You need to know your model structure. This is an example:
target_layer = "maia_4.conv3.0" # e.g., the last conv layer, adjust based on your model

# ----------------------
# Instantiate CAM methods
# ----------------------
cam_methods = {
    "Grad-CAM": GradCAM(model, target_layer),
    "Grad-CAM++": GradCAMpp(model, target_layer),
    "Score-CAM": ScoreCAM(model, target_layer)
}

# Load one image from dataset
img, label = train_dataset[0]
# ----------------------
# Inference and CAM computation
# ----------------------
outputs = model(input_tensor)
pred_class = outputs.argmax(dim=1).item()

# Compute and visualize CAMs
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

for i, (name, cam) in enumerate(cam_methods.items()):
    cam_map = cam(pred_class, outputs)  # This modifies internal state
    heatmap = cam_map[0].cpu()  # [0] since batch_size = 1

    # Overlay CAM on image
    raw_img = transforms.ToPILImage()(input_tensor.squeeze().cpu())
    overlayed = overlay_mask(raw_img, heatmap, alpha=0.5)

    axs[i].imshow(overlayed)
    axs[i].set_title(name)
    axs[i].axis('off')

plt.tight_layout()
plt.show()


ImportError: Matplotlib requires numpy>=1.23; you have 1.21.5